In [ ]:
# pipeline to reproduce 7 day organoid data 

import scanpy as sc
import scanpy.external as sce
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import magic
import matplotlib.colors
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
from gprofiler import gprofiler
from seaborn import despine
from seaborn import axes_style
from matplotlib.pyplot import suptitle

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)
results_file = 'organoids_KY.h5ad'  # the file that will store the analysis results

#load and merge files

filenames=['KY_empty_filtered_feature_bc_matrix.h5','KY_cre_filtered_feature_bc_matrix.h5']

adatas=[sc.read_10x_h5(filename) for filename in filenames]

adata=adatas[0].concatenate(adatas[1:],batch_categories=["KY Empty","KY Cre"])

adata.var_names_make_unique()  # this is unnecessary if using 'gene_ids'

# compute %mito and remove cells with >10% followed by normalization, logging and MAGIC
mito_genes = adata.var_names.str.startswith('mt-')
# for each cell compute fraction of counts in mito genes vs. all genes
# the `.A1` is only necessary as X is sparse (to transform to a dense array after summing)
adata.obs['percent_mito'] = np.sum(adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
# add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = adata.X.sum(axis=1).A1

# plot mito vs count data before filtering
sc.settings.set_figure_params(dpi=80)
with axes_style({'axes.grid': False}):
 sc.pl.scatter(adata, y='percent_mito', x='n_counts', size=5)

# filtering
adata = adata[adata.obs['percent_mito'] < 0.1, :]
sc.pp.filter_genes(adata, min_cells=3)

# plot mito vs count data after filtering
sc.settings.set_figure_params(dpi=80)
with axes_style({'axes.grid': False}):
 sc.pl.scatter(adata, y='percent_mito', x='n_counts', size=5)

sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)

# No consensus about gene scaling so I did not scale the data. Read https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6582955/
# Not scaling argueably retains biological information
# I did not regress cell cycle either. A population of proliferative or non-proliferative cells in this context would be high interesting in my opinion, and is not information
# I want to remove!

adata.raw = adata # save a raw data file

# data diffusion tool is in scanpy.external 
sce.pp.magic(adata, name_list='all_genes', k=5, t=15, n_pca=20)

sc.pp.neighbors(adata, n_neighbors=5, n_pcs=20)
sc.tl.louvain(adata, resolution=0.025, key_added='louvain_r0.025')
sc.tl.louvain(adata, resolution=0.05, key_added='louvain_r0.05')
sc.tl.louvain(adata, resolution=0.1, key_added='louvain_r0.1')
sc.tl.louvain(adata, resolution=0.2, key_added='louvain_r0.2')
sc.tl.louvain(adata, resolution=0.3, key_added='louvain_r0.3')
sc.tl.louvain(adata, resolution=0.4, key_added='louvain_r0.4')
sc.tl.louvain(adata, resolution=0.5, key_added='louvain_r0.5')

# create umap
sc.tl.umap(adata)

# Visualize Louvain resolutions
sc.pl.umap(adata, color=['batch','louvain_r0.025','louvain_r0.05','louvain_r0.1','louvain_r0.2','louvain_r0.3','louvain_r0.4','louvain_r0.5'])


In [ ]:
# Create a PAGA-initalized UMAP
sc.tl.paga(adata, groups='louvain_r0.05')
sc.pl.paga(adata, plot=True, color=['batch'])  # remove `plot=False` if you want to see the coarse-grained graph
# sc.tl.umap(adata, init_pos='paga') is not working (August 7, 2019)
# https://github.com/theislab/scanpy/issues/769 = work around for now..
sc.tl.umap(adata, init_pos=sci.tl._utils.get_init_pos_from_paga(adata))

# Visualize the data 
sc.pl.umap(adata, frameon=False, color=['louvain_r0.05'], legend_loc='right margin') # louvain communities
sc.pl.umap(adata, frameon=False, color=['batch'], legend_loc='right margin') # batch

# Remove small cluster
adata.obs['louvain_r0.05'].value_counts() # count the number of cells per cluster
adata_subset = adata[adata.obs['louvain_r0.05'].isin(['0','1','2','3','4'])] # Remove clusters with < 50 cells


In [ ]:
# contribution of each library to each Louvain cluster
ax = sb.countplot(x="louvain_r0.05", hue="batch", data=adata_subset.obs)
plt.xlabel("Louvain communities")
plt.ylabel("Cells")

In [ ]:

# Calculate gene signatures. Principle is true for all gene lists provided in supplementary. Kras score provided as an example
bild_kras = pd.read_excel('Supplementary Table 2.xlsx', sheet_name='Bild et al. 2006 (Kras genes)', header=0) # load lists 

bild_kras_list = bild_kras['GeneSymbol'].tolist() # Make list of gene names

# Gene names are capitals. Lower and capitalize to be compatible with dataset gene names

bild_kras_list = [x.lower() for x in bild_kras_list] # lower
bild_kras_list = [x.capitalize() for x in bild_kras_list] # capitalize

bild_kras_list_final = [x for x in bild_kras_list if x in adata.var_names] # remove genes not in adata.var

# Calculating z-scores for single cells
# Derived from https://github.com/theislab/scanpy/issues/181

# Create a cell barcodes column
adata_subset.obs['index1'] = adata_subset.obs.index

# create the marker dict 
marker_dict = dict()
marker_dict['Kras z-score'] = bild_kras_list_final

# create the function
def evaluate_partition(anndata, marker_dict, gene_symbol_key=None, partition_key=None):
    # Inputs:
    #    anndata         - An AnnData object containing the data set and a partition
    #    marker_dict     - A dictionary with cell-type markers. The markers should be stores as anndata.var_names or 
    #                      an anndata.var field with the key given by the gene_symbol_key input
    #    gene_symbol_key - The key for the anndata.var field with gene IDs or names that correspond to the marker 
    #                      genes
    #    partition_key   - The key for the anndata.obs field where the cluster IDs are stored. The default is
    #                      'louvain_r1' 

    #Test inputs
    if partition_key not in anndata.obs.columns.values:
        print('KeyError: The partition key was not found in the passed AnnData object.')
        print('   Have you done the clustering? If so, please tell pass the cluster IDs with the AnnData object!')
        raise

    if (gene_symbol_key != None) and (gene_symbol_key not in anndata.var.columns.values):
        print('KeyError: The provided gene symbol key was not found in the passed AnnData object.')
        print('   Check that your cell type markers are given in a format that your anndata object knows!')
        raise
        
    if gene_symbol_key:
        gene_ids = anndata.var[gene_symbol_key]
    else:
        gene_ids = anndata.var_names
        
    # Create a column based on index. This allows z-score calculation on single cells rather than clusters
    clusters = np.unique(anndata.obs[partition_key])
    n_clust = len(clusters)
    n_groups = len(marker_dict)
    
    marker_res = np.zeros((n_groups, n_clust))
    z_scores = sc.pp.scale(anndata, copy=True) # try changing this to anndata_raw as a separate function

    i = 0
    for group in marker_dict:
        # Find the corresponding columns and get their mean expression in the cluster
        j = 0
        for clust in clusters:
            cluster_cells = np.in1d(z_scores.obs[partition_key], clust)
            marker_genes = np.in1d(gene_ids, marker_dict[group])
            marker_res[i,j] = z_scores.X[np.ix_(cluster_cells,marker_genes)].mean()
            j += 1
        i+=1

    variances = np.nanvar(marker_res, axis=0)
    if np.all(np.isnan(variances)):
        print("No variances could be computed, check if your cell markers are in the data set.")
        print("Maybe the cell marker IDs do not correspond to your gene_symbol_key input or the var_names")
        raise

    marker_res_df = pd.DataFrame(marker_res, columns=clusters, index=marker_dict.keys())
    
    return marker_res_df

   # Return the median of all the variances over the clusters
    #marker_matches = ([np.median(variances), marker_res_df])
    
   # return marker_matches

# Calculate the z-score
df = evaluate_partition(adata_subset, marker_dict, gene_symbol_key=None, partition_key = 'index1')

# Transpose the dataframe
df_transposed = df.transpose()

# add score to adata.obs
adata_subset.obs['Kras z-score'] = df_transposed['Kras z-score']

# save the data after adding the z-score
adata_subset.write(results_file)

In [ ]:
# Generate heatmap using Seaborn

d = adata_subset.obs.pivot('louvain_r0.05',"index1", "Kras z-score")
plt.figure(figsize=(5,2))
ax = sb.heatmap(d, cmap='RdYlBu_r', xticklabels=False)
plt.title('Kras score')

In [ ]:
# Mann-Whitney test is for independent variables with no distribution assumptions
from scipy.stats import mannwhitneyu
mannwhitneyu(cat2['Kras score'], cat4['Kras score']) 
mannwhitneyu(cat2['Kras score'], cat0['Kras score']) 
mannwhitneyu(cat2['Kras score'], cat1['Kras score']) 
mannwhitneyu(cat2['Kras score'], cat3['Kras score']) 

In [ ]:
# Perform DE analysis using the in-built ScanPy function
sc.tl.rank_genes_groups(adata_subset, 'louvain_r0.05', method='wilcoxon',n_genes=500, use_raw=True)
result = adata_subset.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame({group + '_' + key[:1]: result[key][group] for group in groups for key in ['names', 'pvals_adj','logfoldchanges']}).head(500)

# export DE results to excel 
df1=pd.DataFrame({group + '_' + key[:1]: result[key][group] for group in groups for key in ['names', 'pvals_adj','logfoldchanges']})
df1.to_excel("organoids_DE_top500.xlsx", sheet_name='Sheet_name_1')

# filtered differential expression for single gene analysis

sc.tl.rank_genes_groups(adata_subset, 'louvain_r0.05', method='wilcoxon',n_genes=1000, use_raw=True)

sc.tl.filter_rank_genes_groups(adata_subset, 
    min_fold_change = 1, # minimum log fold change
    min_in_group_fraction = 0.5,
    max_out_group_fraction = 0.5, 
  key_added='rank_genes_groups_filtered')

result = adata_subset.uns['rank_genes_groups_filtered']
groups = result['names'].dtype.names

# export DE results to excel 
df2=pd.DataFrame({group + '_' + key[:1]: result[key][group] for group in groups for key in ['names', 'pvals_adj','logfoldchanges']})
df2.to_excel("organoids_DE_top1000_filtered.xlsx", sheet_name='filtered results')

sc.tl.rank_genes_groups(adata_subset, 'louvain_r0.05', method='wilcoxon',n_genes=1000, use_raw=True)

result = adata_subset.uns['rank_genes_groups']
groups = result['names'].dtype.names

# export DE results to excel 
organoids_DE_df=pd.DataFrame({group + '_' + key[:1]: result[key][group] for group in groups for key in ['names', 'pvals_adj','logfoldchanges']})
organoids_DE_df.to_excel("organoid_DE_top1000_unfiltered.xlsx", sheet_name='Top 1000 unfiltered')

# Visualize filtered results using a heatmap
heatmap_cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['lightblue','lightyellow','lightcoral'])
sc.pl.rank_genes_groups_heatmap(adata_subset, n_genes=25,cmap=heatmap_cmap, swap_axes=True, use_raw=False, key='rank_genes_groups_filtered', vmin=0, vmax=1.5, show_gene_labels=True)


In [ ]:
# identify TF/TFCs amongst DE genes

# import list of mouse TFs from TFDB
# http://bioinfo.life.hust.edu.cn/AnimalTFDB/#!/species
df_tf = pd.read_excel('Supplementary Table 2.xlsx', sheet_name='TF and TFCs Animal TFDB3', header=0)
tf_genes = set(df_tf['Symbol'].tolist())

# df2 is filtered DE genes from all the organoid clusters

# Find TF/TFCs
names = df2.columns # filtered DE results
clust_dict = {} #key,value

for col in df2.columns:
    
    # get diff gene list
    cluster_genes = df2[col].tolist()

    # Create results array
    results_tfs = []

    # iterate through and pull out common genes
    
    for i in tf_genes:
     if i in cluster_genes:
      results_tfs.append(i)
    
    #store in dict (receptors,ligands,tfs)
    the_key = col
    the_value = {}
    the_value["tfs"] = results_tfs
    clust_dict[the_key] = the_value

# Create a list of identified TFs
merged_tf = []

for i in clust_dict:
    sub_dict = clust_dict[i] # access each cluster in dict
    
    # create merge
    merged_tf += sub_dict["tfs"]

# plot the results using a matrixplot
sc.pl.matrixplot(adata_subset, var_names=merged_tf, cmap='viridis', groupby='louvain_r0.05', use_raw=False, swap_axes=True, vmin=0, vmax=0.6, dendrogram=False)


In [ ]:
# Look for enriched Gene Ontology Biological Process 2018 pathways
%matplotlib inline
%config InlineBackend.figure_format='retina' # mac
import gseapy as gp
from gseapy.plot import barplot, dotplot

#view available reference libraries
names = gp.get_library_name()
print(names)

# turn columns into lists (filtered data)
C0 = organoids_DE_df['0_n']
C1 = organoids_DE_df['1_n']
C2 = organoids_DE_df['2_n']
C3 = organoids_DE_df['3_n']
C4 = organoids_DE_df['4_n']

# drop NaN because they causes Enricher to break
C0=C0.dropna()
C1=C1.dropna()
C2=C2.dropna()
C3=C3.dropna()
C4=C4.dropna()

# GO Analysis
Cluster0_GOBio = gp.enrichr(gene_list = C0,
description='Cluster0_GOBio',
gene_sets=['GO_Biological_Process_2018'],
outdir='Enricher_analysis/Cluster0_GOBio',
cutoff=0.05
)

Cluster1_GOBio = gp.enrichr(gene_list = C1,
description='Cluster1_GOBio',
gene_sets=['GO_Biological_Process_2018'],
outdir='Enricher_analysis/Cluster1_GOBio',
cutoff=0.05
)

Cluster2_GOBio = gp.enrichr(gene_list = C2,
description='Cluster2_GOBio',
gene_sets=['GO_Biological_Process_2018'],
outdir='Enricher_analysis/Cluster2_GOBio',
cutoff=0.05
)

Cluster3_GOBio = gp.enrichr(gene_list = C3,
description='Cluster3_GOBio',
gene_sets=['GO_Biological_Process_2018'],
outdir='Enricher_analysis/Cluster3_GOBio',
cutoff=0.05
)

Cluster4_GOBio = gp.enrichr(gene_list = C4,
description='Cluster4_GOBio',
gene_sets=['GO_Biological_Process_2018'],
outdir='Enricher_analysis/Cluster4_GOBio',
cutoff=0.05
)

#  Find unique GO terms

# filter for statistically significant terms
c0_sig = Cluster0_GOBio.res2d.loc[(Cluster0_GOBio.res2d['Adjusted P-value'] < 0.05)] 
c1_sig = Cluster1_GOBio.res2d.loc[(Cluster1_GOBio.res2d['Adjusted P-value'] < 0.05)] 
c2_sig = Cluster2_GOBio.res2d.loc[(Cluster2_GOBio.res2d['Adjusted P-value'] < 0.05)] 
c3_sig = Cluster3_GOBio.res2d.loc[(Cluster3_GOBio.res2d['Adjusted P-value'] < 0.05)] 
c4_sig = Cluster4_GOBio.res2d.loc[(Cluster4_GOBio.res2d['Adjusted P-value'] < 0.05)] 

# create lists
c0_list = c0_sig['Term'].tolist() # cancer
c1_list = c1_sig['Term'].tolist() # cancer
c2_list = c2_sig['Term'].tolist() # wt
c3_list = c3_sig['Term'].tolist() # cancer
c4_list = c4_sig['Term'].tolist() # wt


# find unique GO terms
c0_unique_terms = []
c1_unique_terms = []
c2_unique_terms = []
c3_unique_terms = []
c4_unique_terms = []
common_terms = []

for i in c0_list:
    if i not in c1_list and i not in c2_list and i not in c3_list and i not in c4_list:
        c0_unique_terms.append(i)
        
for i in c1_list:
    if i not in c0_list and i not in c2_list and i not in c3_list and i not in c4_list:
        c1_unique_terms.append(i)
        
for i in c2_list:
    if i not in c0_list and i not in c1_list and i not in c3_list and i not in c4_list:
        c2_unique_terms.append(i)
        
for i in c3_list:
    if i not in c0_list and i not in c1_list and i not in c2_list and i not in c4_list:
        c3_unique_terms.append(i)
        
for i in c4_list:
    if i not in c0_list and i not in c1_list and i not in c2_list and i not in c3_list:
        c4_unique_terms.append(i)
        
# find common GO terms. Note: None were found in my dataset
for i in c0_list:
    if i in c1_list and i in c2_list and i in c3_list and i in c4_list:
        common_terms.append(i)

In [ ]:
# Sox9 activation and expression in single cells

TTRUST_sox9 = pd.read_excel('Supplementary Table 2.xlsx', sheet_name='Sox9 targets TRRUST', header=0) # load data
TTRUST_activation_sox9 = TTRUST_sox9.loc[(TTRUST_sox9['Type'] == 'Activation')] # Extract gene defined as activated
TTRUST_activation_sox9_list = TTRUST_activation_sox9['Target'].tolist() # create list
TTRUST_activation_sox9_list = [x for x in TTRUST_activation_sox9_list if x in adata.var_names] # remove genes not in adata_var

# Calculating z-scores for single cells

# create the marker dict 
marker_dict = dict()
marker_dict['Sox9 activation z-score'] = TTRUST_activation_sox9_list

# Calculate the z-score
# 'evaluate_partition' function was defined earlier
df = evaluate_partition(adata_subset, marker_dict, gene_symbol_key=None, partition_key = 'index1')

# Transpose the dataframe
df_transposed = df.transpose()
adata_subset.obs['Sox9_score'] = df_transposed['Sox9 activation z-score']

# save the data
adata_subset.write(results_file)

# add Sox9 expression to adata_subset.obs for TF/activity visualization
adata_subset.obs['Sox9']=adata_subset[:, ['Sox9']].to_df()

scores = adata_subset.obs[['Klf4_score','Myc_score','Pou5f1_score','Sox2_score','Sox9_score','Sox2','Myc','Pou5f1','Klf4','Sox9']]
sox9_phase = pd.Series('No activity', index=scores.index) # default state
sox9_phase[(scores.Sox9 > 0) & (scores.Sox9_score < 0)] = 'sox9 expression only'
sox9_phase[(scores.Sox9 < 0) & (scores.Sox9_score > 0)] = 'sox9 activity only'
sox9_phase[(scores.Sox9 > 0) & (scores.Sox9_score > 0)] = 'sox9 expression + activity'

# Add information to adata_subset
adata_subset.obs['sox9_phase'] = sox9_phase


In [ ]:
#### You can download the annotated adata_subset file and begin data analysis straight away! ####

# load adata_subset file shown in the figures
results_file = 'adata_subset.h5ad'  
adata_subset = sc.read(results_file)

In [ ]:
# Hope you find the data and code useful! :)